# Bifurcation equations of the perfect system {#sec-20230102153124}

In this notebook, the bifurcation analysis of the perfect system is performed symbolically. This notebook uses custom $\LaTeX\newcommand{\D}{\mathrm{d}}\newcommand{\E}{\mathcal{E}}\newcommand{\reals}{\mathbb{R}}$ macros.

Symbolic expressions of the energy $(u, \lambda) \mapsto \E(u, \lambda)$ and of the fundamental path $\lambda \mapsto u^\ast(\lambda)$ are first constructed in @sec-20230103084732. These expressions are Taylor expansions about the critical point $(u_0, \lambda_0)$.

Without loss of generality, it will be assumed in this notebook that $\lambda_0 = 0$ and $u_0 = 0$. The general case $\lambda_0 \neq 0$ and $u_0 \neq 0$ is readily recovered through the substitution $\lambda \leftrightarrow \lambda - \lambda_0$ and $u \leftrightarrow u - u_0$.

## Initialization

We use the [Python](https://www.python.org) library [SymPy](https://www.sympy.org) (imported as `sp`, here) for symbolic mathematics. We define a few common symbols and useful functions.

In [1]:
import sympy as sp

def latex(x):
    return x if str(x) == x else sp.latex(x)
  
zero = sp.Integer(0)

λ, u, v, u_hat, v_hat, w_hat = sp.symbols(r"\lambda u v \hat{u} \hat{v} \hat{w}")

The [IPython](https://ipython.org/) library (in particular, [IPython.display.Math](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html#IPython.display.Math)) will be used for pretty $\LaTeX$ output.

In [2]:
import IPython.display as ipd

def display_latex_str(s):
    display(ipd.Math(s))

def display_latex_dict(d, num_cols=2):
    s = "\\begin{align}"
    col = 1
    for k, v in d.items():
        s += latex(k) + "&=" + latex(v)
        s += r"\\" if col % num_cols == 0 else r"&"
        col += 1
    s += "\\end{align}"
    display_latex_str(s)

def display_latex_equation(lhs, rhs):
    display_latex_str(latex(lhs) + "=" + latex(rhs))

## Symbolic expressions of the energy and fundamental path {#sec-20230103084732}

The following developments involve the energy $(u, \lambda) \mapsto \E(u, \lambda)$ and its differentials at the critical point $(u_0, \lambda_0)$, as well as the fundamental path $\lambda \mapsto u^\ast(\lambda)$ and its derivatives at $\lambda = \lambda_0$. It will therefore be convenient to express $\E$ and $u^\star$ as Taylor expansions with respect to $u$ and $\lambda$.

We start with the Taylor expansion of the energy $\E$. We define its differentials at the critical point. These differentials are stored in a dictionary. Values are indexed with the order of the differentials with respect to $u$ and $\lambda$.

In the following code blocks, `n` is the total order of the differential, `p` is the order of the differential with respect to $u$ and `q = n - p` is the order of the differential with respect to $\lambda$.

In [3]:
dE = dict()
for n in range(1, 5):
    for p in range(n + 1):
        q = n - p
        if q == 0:
            dE[p, q] = sp.Symbol(r"\E_{}".format(p))
        else:
            dE[p, q] = sp.Symbol(r"\E_{" + p * "u" + q * "\lambda" + "}")
            
# Remember that the energy is stationary at the critical point (equilibrium)
dE[1, 0] = zero

The dictionary `dE` should be understood as follows: `dE[(2, 1)] == dE[2, 1]` is the symbol $\E_{uu\lambda}$, which stands for $\E_{,uu\lambda}(u_0, \lambda_0; \bullet, \bullet)$. The following symbols have been defined.

In [4]:
#| code-fold: true

_dict = dict()
for n in range(1, 5):
    for p in range(n + 1):
        q = n - p
        _dict[dE[p, q]] = ("\\E_{," + p * "u" 
                           + q * "\\lambda"
                           + "}(u_0, \lambda_0)")
        
display_latex_dict(_dict, num_cols=3)

<IPython.core.display.Math object>

where

- $\E_\lambda$, $\E_{\lambda\lambda}$, $\E_{\lambda\lambda\lambda}$ and $\E_{\lambda\lambda\lambda\lambda}$ are *scalar quantities*,
- $\E_{u\lambda}$, $\E_{u\lambda\lambda}$ and $\E_{u\lambda\lambda\lambda}$ are *linear forms*,
- $\E_2$, $\E_{uu\lambda}$ and $\E_{uu\lambda\lambda}$ are *bilinear forms*,
- $\E_3$ and $\E_{uuu\lambda}$ are *trilinear forms*,
- $\E_4$  is a *quadrilinear form*.

Note that all these differentials are defined as SymPy *scalars*. A definition as a SymPy *function* (e.g. `E2 = Function(r"\E_2")`) would be more appropriate. However, SymPy would fail to account for multilinearity or symmetry of these forms. Therefore, we use the following trick: all multi-linear forms are defined as scalars, and the standard multiplication operator `*` means function application. In other words, `E2 * (α * u + β * v) * w` (resp. `E2 * u * v - E2 * v * u`) should be understood as `E2(α * u + β * v, w)` (resp. `E2(u, v) - E2(v, u)`). In both cases, the expressions would are correctly simplified.

Whether the symbols in an expression are true scalars or vectors (elements of $U$) should be clear from the context. For example, in the expression: `λ * E2 * u * v`, the first `*` is a true multiplication, while the other `*` refer to function application.

The energy $\E(u, \lambda)$ is now expressed as a Taylor expansion about the critical point.

In [5]:
_f = lambda p, q: sp.binomial(p + q, p) / sp.factorial(p + q)
E = sum(_f(p, q) * dE_pq * u**p * λ**q for (p, q), dE_pq in dE.items())

In [6]:
#| code-fold: true
display_latex_equation(r"\E(u, \lambda)", E)

<IPython.core.display.Math object>

We now turn to the definition of the fundamental path $\lambda \mapsto u^\star(\lambda)$. We first define in a dictionary the derivatives of $u^\star$ with respect to $\lambda$, at $\lambda = \lambda_0 = 0$. For example, `du_0[2]` is the symbol $\ddot{u}_0$, which stands for $\D^2 u^\star / \D \lambda^2 \rvert_{\lambda=\lambda_0}$.

In [7]:
du_0 = {n: sp.Symbol("\\" + "".join(n * ("d",)) + "ot{u}_0") for n in range(1, 5)}

_dict = {f"\\frac{{\\D^{k}u^\\ast}}{{\\D \\lambda^{k}}}" : v for k, v in du_0.items()}

In [8]:
#| code-fold: true
display_latex_dict(_dict, num_cols=4)

<IPython.core.display.Math object>

The fundamental path is then defined through its Taylor expansion.

In [9]:
u_star = sum(λ**n / sp.factorial(n) * v for n, v in du_0.items())

In [10]:
#| code-fold: true
display_latex_equation(r"u^\star(\lambda)", u_star)

<IPython.core.display.Math object>

It will be assumed in the following that $v, \hat{v} \in V$, therefore terms of the form $\E_2(\bullet, v)$ and $\E_2(\bullet, \hat{v})$ should vanish. The following function performs this simplification.

In [11]:
# Let's define a convenient alias to the Hessian
E2 = dE[2, 0]

def simplify_E2(expr):
    expr = expr.expand()
    coeff_E2 = expr.coeff(E2).expand()
    coeff_E2_v = coeff_E2.coeff(v)
    coeff_E2_v_hat = (coeff_E2 - coeff_E2_v * v).expand().coeff(v_hat)
    return (expr - (coeff_E2_v * E2 * v + coeff_E2_v_hat * E2 * v_hat)).expand()

assert simplify_E2(E2 * v) == 0
assert simplify_E2(E2 * v_hat) == 0
assert simplify_E2(3 * v + 4 * v_hat + 5 * v * v_hat + 6 * E2 * v + 7 * E2 * v_hat + 8 * E2 * v * v_hat) == 3 * v + 4 * v_hat + 5 * v * v_hat

## Elimination of the mixed derivatives of the energy

Since the fundamental path $\lambda \mapsto u^\ast(\lambda)$ is an equilibrium path, the various differentials of the energy at the critical point are not linearly independent. To express the relationships between these forms, we define $\mathcal R^\ast(\lambda; \bullet)$ as the jacobian of the energy along the fundamental path $u^\ast(\lambda)$

$$
\mathcal R^\ast(\lambda; \bullet) = \E_{,u}[u^\ast(\lambda), \lambda; \bullet].
$$

In [12]:
R_star = (E.diff(u) * u_hat).subs(u, u_star).expand()

In [13]:
#| code-fold: true
display_latex_equation(r"\mathcal{R}^\ast(\lambda;" + sp.latex(u_hat) + ")",
                       R_star.series(λ, 0, 5))

<IPython.core.display.Math object>

Of course, since $\lambda \mapsto u^\ast(\lambda)$ is an equilibrium path, we have $\mathcal R^\ast(\lambda; \bullet) = 0$ for all $\lambda$. Therefore, all coefficients of the above polynomial in $\lambda$ are null, which delivers expressions of $\E_{u\lambda}$, $\E_{u\lambda\lambda}$ and $\E_{u\lambda\lambda\lambda}$

In [14]:
mixed1 = dict()

for q in range(1, 4):
    x = dE[1, q]
    eq = R_star.coeff(λ, q)
    sol = sp.solve(eq, x)[0]
    mixed1[x] = sol

In [15]:
#| code-fold: true
display_latex_dict(mixed1, num_cols=1)

<IPython.core.display.Math object>

We want to get rid of the remaining mixed derivatives, namely: $\E_{uu\lambda}$, $\E_{uuu\lambda}$ and $\E_{uu\lambda\lambda}$. To do so, we introduce the derivatives $\dot{\E}_2$, $\ddot{\E}_2$ and $\dot{\E}_3$ defined in @sec-20230105101657.

In [16]:
E2_dot, E2_ddot, E3_dot = sp.symbols("\dot{\E}_2 \ddot{\E}_2 \dot{\E}_3")

E_uu_star = E.diff(u, 2).subs(u, u_star).expand()
E_uuu_star = E.diff(u, 3).subs(u, u_star).expand()

eqs = [sp.Eq(E2_dot, E_uu_star.coeff(λ, 1)),
       sp.Eq(E3_dot, E_uuu_star.coeff(λ, 1)),
       sp.Eq(E2_ddot, 2 * E_uu_star.coeff(λ, 2))]

unknowns = [dE[2, 1], dE[3, 1], dE[2, 2]] # E_uuλ, E_uuuλ, E_uuλλ

mixed2 = dict()
for eq, x in zip(eqs, unknowns):
    mixed2[x] = sp.solve(eq, x)[0].subs(mixed2).expand()

We get the following expressions

In [17]:
#| code-fold: true
display_latex_dict(mixed2, num_cols=1)

<IPython.core.display.Math object>

Combining the above results allows to fully eliminate the mixed derivatives

In [18]:
#| code-fold: true

mixed = {k: v.subs(mixed2).expand() for k, v in mixed1.items()}
mixed.update(mixed2)

display_latex_dict(mixed, num_cols=1)

<IPython.core.display.Math object>

These expressions can be plugged into the expansion of the energy.

In [19]:
E = E.subs(mixed).expand()

In [20]:
#| code-fold: true
display_latex_equation(r"\E(u, \lambda)", E)

<IPython.core.display.Math object>

## Derivation of the bifurcation equations

In this section, we show that, besides the fundamental branch $u^\ast(\lambda)$, other (bifurcated) equilibrium branches
may pass through the critical point $(u_0, \lambda_0)$. The starting point is the characterization of an equilibrium by
the stationarity of the energy, which defines all equilibrium branches as implicit functions, which can be expanded with
respect to some perturbation parameter.

The approach adopted here relies on the Lyapunov--Schmidt decomposition of the equilibrium branch over $V$ and
$W$. Other approaches are possible, see e.g. @chak2018.

For a given value of $\lambda$ (close to the critical value $\lambda_0$), We seek an equilibrium point $u$ of the
system, such that

$$
\E_{,u}(u, \lambda; \hat{u}) = 0 \quad \text{for all} \quad \hat{u} \in U.
$$ {#eq-20230102025737}

In [21]:
res = (E.diff(u) * u_hat).expand()

The above equation reads: for all $\hat{u} \in U$

In [22]:
#| code-fold: true
display_latex_equation(res, 0)

<IPython.core.display.Math object>

The equilibrium state $u$ is projected onto the orthogonal subspaces $V$ and $W$.

$$
u = u^\ast(\lambda) + \xi_i \, v_i + w, \quad \text{with} \quad w \in W.
$$ {#eq-20220902174235}

It follows from the orthogonality of $V$ and $W$ that $\langle v_i, w \rangle = 0$ for all $i=1, \ldots, m$. Analysis of the bifurcated branches therefore reduces to finding $\xi_1, \ldots, \xi_m \in \reals$ and $w \in W$, such that

$$
\E_{,u}[u^\ast(\lambda) + \xi_i \, v_i + w, \lambda; \hat{u}] = 0 \quad \text{for all} \quad \hat{u} \in U.
$$ {#eq-20230107164346}

The method proceeds in three steps

1. Eq. (@eq-20230107164346) is first expressed for $\hat{u} \in W$ which leads to a definition of $w$ as an implicit function of $\xi_1, \ldots, \xi_m$ and $\lambda$ (see @sec-20230102030010),
2. Eq. (@eq-20230107164346) is then expressed for $\hat{u} \in V$ which leads to a definition of $\lambda$ as an implicit function of $\xi_1, \ldots, \xi_m$ (see @sec-20230102030125),
3. finally, a parametrization $\eta$ of $\xi_1$, \dots $\xi_m$ is introduced and the Taylor expansion of $u$ and $\lambda$ with respect to $\eta$ is derived (see @sec-20230102030224).

### Elimination of $w$ {#sec-20230102030010}

We first test Eq. (@eq-20230102025737) with $\hat{w} \in W$

$$
\E_{,u}[u^\ast(\lambda) + \xi_i \, v_i + w, \lambda; \hat{w}] = 0, \quad \text{for all} \quad \hat{w} \in W.
$$ {#eq-20220825143616}

This defines $w$ as an implicit function of $\xi_1, \ldots, \xi_m$ and $\lambda$. The following asymptotic expansion
about $\xi_1 = \cdots = \xi_m = 0$ and $\lambda = \lambda_0$ is postulated

$$
\begin{aligned}
w(\xi_1, \ldots, \xi_m, \lambda) ={} & \xi_i \, w_i + \bigl( \lambda - \lambda_0 \bigr) \, w_\lambda\\
&+ \tfrac{1}{2} \xi_i \, \xi_j \, w_{ij} + \tfrac{1}{2} \bigl( \lambda - \lambda_0 \bigr)^2 w_{\lambda\lambda} + \xi_i \, \bigl( \lambda - \lambda_0 \bigr) w_{i\lambda} + \cdots,
\end{aligned}
$$ {#eq-20230107172048}

where $w_i$, $w_\lambda$, $w_{ij}$, $w_{\lambda\lambda}$ and $w_{i\lambda}$ are elements of $W$ to be defined.

Indicial expressions are somewhat difficult to handle in SymPy, we therefore use a "trick"

- `ξ` stands for $\xi_i$,
- `w_ξ` stands for $w_i$,
- `w_ξξ`stands for $w_{ij}$,
- `w_ξλ` stands for $w_{i\lambda}$.

Then expressions such as `ξ * w_ξ` (resp. `ξ^2 * w_ξξ`) should be understood as $\xi_i \, w_i$ (resp. $\xi_i \, \xi_j \, w_{ij}$).

In [23]:
ξ, w_ξ, w_λ, w_ξξ, w_ξλ, w_λλ = sp.symbols(r"\xi w_\xi w_\lambda w_{\xi\xi} w_{\xi\lambda} w_{\lambda\lambda}")
w = ξ * w_ξ + λ * w_λ + (ξ**2 * w_ξξ + 2 * ξ * λ * w_ξλ + λ**2 * w_λλ) / 2

It will be useful to define the critical point

In [24]:
crit = {ξ: 0, λ: 0}

Eq. (@eq-20230107172048) is plugged into Eq. (@eq-20220825143616), which then delivers an asymptotic expansion of the stationarity equation. We will nullify the terms of order up to 2 in $\xi_i$ and $\lambda$.

In [25]:
res_w = res.subs({u_hat: w_hat, u: u_star + ξ * v + w}).expand()

We can then nullify the various coefficients of this polynomial expansion, starting with the $\xi_i$ terms

In [26]:
eq = sp.Eq(simplify_E2(res_w.coeff(ξ)), 0).subs(crit)
display(eq)

Eq(\E_2*\hat{w}*w_\xi, 0)

which should be understood as

$$
\E_2(w_i, \hat{w}) = 0 \quad \text{for all} \quad \hat{w} \in W.
$$

As argued in @sec-20230107173921, this leads to $w_i = 0$. We now consider the $\lambda$ term

In [27]:
eq = sp.Eq(simplify_E2(res_w.coeff(λ)), 0).subs(crit)
display(eq)

Eq(\E_2*\hat{w}*w_\lambda, 0)

Which sould be understood as

$$
\E_2(w_\lambda, \hat{w}) = 0 \quad \text{for all} \quad \hat{w} \in W.
$$

This again leads to $w_\lambda = 0$. The $\lambda^2$ term again delivers then vanishes automatically.

$$
\E_2(w_{\lambda\lambda}, \hat{w}) = 0 \quad \text{for all} \quad \hat{w} \in W,
$$

from which it results that $w_{\lambda\lambda} = 0$

In [28]:
assert 2 * res_w.coeff(λ, 2).subs(crit).subs(w_λ, 0).expand() == E2 * w_λλ * w_hat

The $\xi_i \, \xi_j$ terms deliver the following equations

In [29]:
(eq_w_ξξ := sp.Eq(2 * res_w.coeff(ξ, 2), 0).subs(crit).subs(w_ξ, 0).expand())

Eq(\E_2*\hat{w}*w_{\xi\xi} + \E_3*\hat{w}*v**2, 0)

The above equation should be understood as the following variational problem: find $w_{ij} \in W$ such that

$$
\E_2(w_{ij}, \hat{w}) + \E_3(v_i, v_j, \hat{w}) = 0 \quad \text{for all} \quad \hat{w} \in W,
$$ {#eq-20230107180410}

which defines uniquely the $w_{ij}$. Finally, the $\xi_i \, \lambda$ terms deliver the following variational problem

In [30]:
(eq_w_ξλ := sp.Eq(res_w.coeff(ξ).coeff(λ), 0).subs(crit).subs({w_ξ: 0, w_λ: 0}).expand())

Eq(\E_2*\hat{w}*w_{\xi\lambda} + \dot{\E}_2*\hat{w}*v, 0)

In other words: find $w_{i\lambda} \in W$ such that

$$
\E_2(w_{i\lambda}, \hat{w}) + \dot{\E}_2(v_i, \hat{w}) = 0 \quad \text{for all} \quad \hat{w} \in W,
$$ {#eq-20230107180501}

which again defines $w_{i\lambda}$ uniquely.

As a conclusion, it has been shown that $w_1 = \cdots = w_m = w_\lambda = w_{\lambda\lambda} = 0$. The Taylor expansion of $w$ therefore reduces to

$$
w(\xi_1, \ldots, \xi_m, \lambda) = \tfrac{1}{2} \xi_i \, \xi_j \, w_{ij} + \xi_i \, \bigl( \lambda - \lambda_0 \bigr) w_{i\lambda} + \cdots,
$$

where $w_{ij}$ and $w_{i\lambda}$ are the solutions to the variational problems (@eq-20230107180410) and (@eq-20230107180501), respectively.

### Elimination of $\lambda$ {#sec-20230102030125}

We now test Eq. (@eq-20230102025737) with $\hat{v} \in V$

$$
\E_{,u}[u^\ast(\lambda) + \xi_i \, v_i + w(\xi_1, \ldots, \xi_m, \lambda), \lambda; \hat{v}] = 0, \quad \text{for all} \quad \hat{v} \in W.
$$ {#eq-20230110092313}

This now defines $\lambda$ as an implicit function of $\xi_1, \ldots, \xi_m$. The following asymptotic expansion
about $\xi_1 = \cdots = \xi_m = 0$ is postulated

$$
\lambda(\xi_1, \ldots, \xi_m) = \xi_i \, \lambda_i + \tfrac{1}{2} \, \xi_i \, \xi_j \, \lambda_{ij}\\
$$ {#eq-20230110091031}

where $\lambda_i$ and $\lambda_{ij}$ are scalar coefficients to be defined.

As mentioned above, handling of indicial expressions is a bit awkward in [SymPy](https://www.sympy.org/). Therefore, we define well-formed products such as $\lambda_i \, \xi_i$ or $\lambda_{ij} \, \xi_i \, \xi_j$ as *fully parenthesized symbols*.

The auxiliary variable `t` records the order of each term with respect to the $\xi_1, \ldots, \xi_m$.

In [31]:
t = sp.Symbol("t")

We first introduce the asymptotic expansion, of $(\xi_1, \ldots, \xi_m, \lambda) \mapsto w(\xi_1, \ldots, \xi_m, \lambda)$, $(\xi_1, \ldots, \xi_m) \mapsto \lambda(\xi_1, \ldots, \xi_m)$ and finally $(\xi_1, \ldots, \xi_m) \mapsto u(\xi_1, \ldots, \xi_m)$ see Eqs. (@eq-20230107172048), (@eq-20230110091031) and (@eq-20220902174235).

In [32]:
w_bif = (t * λ * sp.Symbol(r"\bigl(\xi_k\,w_{k\lambda}\bigr)") 
         + t **2 / 2 * sp.Symbol(r"\bigl(\xi_i\,\xi_j\,w_{ij}\bigr)"))

λ_bif = (t * sp.Symbol(r"\bigl(\xi_{p}\,\lambda_{p}\bigr)")
         + t**2 / 2 * sp.Symbol(r"\bigl(\xi_m\,\xi_n\,\lambda_{mn}\bigr)"))

u_bif = u_star + t * sp.Symbol(r"\bigl(\xi_{l}\,v_{l}\bigr)") + w_bif

In [33]:
#| code-fold: true
display_latex_equation(r"w(\xi_1, \ldots, \xi_m, \lambda)", w_bif)
display_latex_equation(r"\lambda(\xi_1, \ldots, \xi_m)", λ_bif)
display_latex_equation(r"u(\xi_1, \ldots, \xi_m)", u_bif)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The above expansions are plugged in the general expression of $\E_{,u}(u, \lambda; \bullet)$ and tested with $\hat{v} \in V$, see Eq. (@eq-20230110092313).

In [34]:
res_v = res.subs(u_hat, v_hat).subs(u, u_bif).subs(λ, λ_bif).expand()

We used three consecutive `subs()` rather than one big dictionary substitution for better control on the *order* of the substitutions. Indeed, we must make sure `λ_bif` is also plugged into `u_bif`. 

We get an asymptotic expansion of the residual with respect to $\xi_1, \ldots, \xi_m$. This residual must vanish for all $\xi_i$ sufficiently small. Therefore, all coefficients of the expansion `res_v` must be nullified at all orders and for all $\xi_i$ sufficiently small. Consiering the expansion up to third order, we get after simplification only *two* bifurcation equations, since the terms of order 0 and 1 are identically null.

In [35]:
coeffs = [simplify_E2(res_v.coeff(t, k)) for k in range(4)]
    
assert coeffs[0] == 0
assert coeffs[1] == 0

In [36]:
#| code-fold: true
for coeff in coeffs[2:]:
    display_latex_equation(coeff, 0)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The above bifurcation equations should be understood as follows
$$
\xi_j \, \xi_k \, \bigl[
\tfrac{1}{2} \E_3(v_j, v_k, \hat{v})
+ \lambda_k \, \dot{\E}_2(v_j, \hat{v})
\bigr] = 0
$$ {#eq-20230115101130}
and
$$
\begin{aligned}[b]
\xi_j \, \xi_k \, \xi_l \, \bigl[
\tfrac{1}{2} \E_3(v_j, w_{kl}, \hat{v})
+ \lambda_l \, \E_3(v_j, w_{k\lambda}, \hat{v})
+ \tfrac{1}{6} \E_4(v_j, v_k, v_l, \hat{v}) &\\
+ \tfrac{1}{2} \lambda_l \, \dot{\E}_2(w_{jk}, \hat{v})
+ \lambda_k \, \lambda_l \, \dot{\E}_2(w_{j\lambda}, \hat{v})
+ \tfrac{1}{2} \lambda_{kl} \, \dot{\E}_2(v_j, \hat{v}) &\\
+ \tfrac{1}{2} \lambda_l \, \dot{\E}_3(v_j, v_k, \hat{v})
+ \tfrac{1}{2} \lambda_k \, \lambda_l \, \ddot{\E}_2(v_j, \hat{v})
\bigr] &= 0.
\end{aligned}
$$ {#eq-20230115101147}

Note that these equations are in fact *asymptotic expansions*. As such, they should hold for all $\xi_1, \ldots, \xi_m$ sufficiently small. Since both equations are homogeneous in $\xi_1, \ldots, \xi_m$, it is equivalent to require these to hold for all $\xi_1, \ldots, \xi_m \in \reals$. 